## 📘 How to Use Kaggle (Upload Dataset & Notebook)

### ✅ Step 1: Create Kaggle Account
- Go to 👉 https://www.kaggle.com  
- Sign in using Google / Email

---

### ✅ Step 2: Upload Your Dataset
1. Click **Datasets** → **Create New Dataset**
2. Upload your **dataset folder or ZIP file**
3. Add:
   - Dataset name
   - Short description
4. Set visibility → **Public / Private**
5. Click **Create**

✅ After upload, Kaggle gives a dataset path like:


## 📞 Customer Churn Prediction using XGBoost (Telecom Dataset)

### 📌 Project Overview

This project builds a **machine learning classification model** to predict whether a telecom customer will **churn (leave the service)** using:

-  XGBoost (Extreme Gradient Boosting)
-  Label Encoding for categorical features
-  Train-Test splitting with stratification
-  Model evaluation using Accuracy & Classification Report
-  Saving trained models & encoders for deployment

The dataset used is the **Telecom Churn Dataset** from Kaggle.

---

#### ✅ Core Technologies (Official Websites)

- XGBoost → https://xgboost.readthedocs.io  
- scikit-learn → https://scikit-learn.org  
- SHAP → https://shap.readthedocs.io  
- Pandas → https://pandas.pydata.org  
- NumPy → https://numpy.org  
- Joblib → https://joblib.readthedocs.io  
- Kaggle → https://www.kaggle.com  


In [ ]:
!pip install shap xgboost scikit-learn

### 🟢 Cell 0: Install Required Machine Learning Libraries

This cell installs all the libraries required for:

- Data preprocessing
- Model training
- Model evaluation
- Model explainability (SHAP)

---

#### ✅ Command Used


SHAP → Model explainability
https://shap.readthedocs.io

XGBoost → High-performance gradient boosting
https://xgboost.readthedocs.io

scikit-learn → ML utilities, preprocessing & evaluation
https://scikit-learn.org


In [ ]:
# -----------------------------
# 1. Import libraries
# -----------------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib
import os



Dataset Path: https://www.kaggle.com/datasets/huebitsvizg/customer-churn-dataset

In [ ]:
# -----------------------------
# 2. Load dataset
# -----------------------------
df = pd.read_csv("/kaggle/input/telecom-churn-datasets/churn-bigml-80.csv")  # full dataset

df.head()


#### 🟢 Cell : Encode Categorical Features using LabelEncoder

Machine learning models require **numerical input**, so we encode the following categorical columns:

In [ ]:
# -----------------------------
# 2. Columns to encode
# -----------------------------
categorical_cols = ["State","International plan","Voice mail plan"]

encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le


#### 🟢 Cell : Save Label Encoders for Deployment

This section saves each trained encoder using **Joblib**.

In [ ]:
# -----------------------------
# 3. Save all encoders
# -----------------------------
save_path = "//kaggle/working/"

for col, le in encoders.items():
    joblib.dump(le, save_path + f"{col}_encoder.pkl")

print("Encoders saved successfully")


In [ ]:
# -----------------------------
# 4. Prepare data & Train XGBoost
# -----------------------------
X = df.drop("Churn", axis=1)
y = df["Churn"].astype(int)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05
)

model.fit(x_train, y_train)


In [ ]:
# -----------------------------
# 5. Evaluate
# -----------------------------

y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))



In [ ]:
import joblib

model_path = "/kaggle/working/xgb_telco_model_final.pkl"
joblib.dump(model, model_path)

print("Model saved at:", model_path)


In [ ]:
import os

for f in os.listdir("/kaggle/working"):
    print(f)


# Run this in colab

## 🟢 Cell: Install Flask & ngrok + Create Project Folders

This cell prepares the **web deployment environment** by:

- Installing the required Python packages for:
  - Backend web server (Flask)
  - Public URL tunneling (ngrok)
- Creating the required project directories for:
  - HTML templates
  - Static files (CSS, images, uploads)

---

In [ ]:
!pip install flask pyngrok

##### Creating folders for fronted

In [ ]:
!mkdir -p templates static

###  Cell: Mount Google Drive in Google Colab

This cell connects your **Google Drive storage** to the Colab environment so that you can:

-  Load trained models
-  Access datasets
-  Save trained outputs
-  Persist files even after the session ends

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##  Cell: Flask Backend for Customer Churn Prediction (app.py)

This cell creates the **Flask web application backend** that:

-  Loads the trained **XGBoost churn model**
-  Loads all saved **Label Encoders**
-  Accepts user input through a web form
-  Preprocesses inputs in the correct feature order
-  Predicts whether a customer will **churn or not**
-  Displays the prediction result with probability

In [ ]:
%%writefile app.py
import os
import joblib
import numpy as np
import pandas as pd
from flask import Flask, render_template, request, redirect, url_for

BASE = "/content/drive/My Drive/Customer churn_hybrid/"

model = joblib.load(BASE + "xgb_telco_model_final.pkl")
state_enc = joblib.load(BASE + "State_encoder.pkl")
intl_enc = joblib.load(BASE + "International plan_encoder.pkl")
voice_enc = joblib.load(BASE + "Voice mail plan_encoder.pkl")

CORRECT_ORDER = [
    "State","Account length","Area code","International plan","Voice mail plan",
    "Number vmail messages","Total day minutes","Total day calls","Total day charge",
    "Total eve minutes","Total eve calls","Total eve charge","Total night minutes",
    "Total night calls","Total night charge","Total intl minutes","Total intl calls",
    "Total intl charge","Customer service calls"
]

app = Flask(__name__, template_folder="templates", static_folder="static")

@app.route("/")
def home():
    return render_template("home.html")

@app.route("/input", methods=["GET", "POST"])
def input_page():

    return render_template("input.html",
                           states=state_enc.classes_,
                           yes_no=intl_enc.classes_,
                           area_codes=["408","415","510"])

@app.route("/predict", methods=["POST"])
def predict():
    try:
        raw = {f: request.form.get(f) for f in CORRECT_ORDER}

        if any(v == "" or v is None for v in raw.values()):
            return render_template("input.html",
                error="⚠ Please fill all fields!",
                states=state_enc.classes_,
                yes_no=intl_enc.classes_,
                area_codes=["408","415","510"])

        X = {}
        X["State"] = state_enc.transform([raw["State"]])[0]
        X["International plan"] = intl_enc.transform([raw["International plan"]])[0]
        X["Voice mail plan"] = voice_enc.transform([raw["Voice mail plan"]])[0]

        numeric_fields = [
            "Account length","Area code","Number vmail messages","Total day minutes",
            "Total day calls","Total day charge","Total eve minutes","Total eve calls",
            "Total eve charge","Total night minutes","Total night calls",
            "Total night charge","Total intl minutes","Total intl calls",
            "Total intl charge","Customer service calls"
        ]

        for f in numeric_fields:
            X[f] = float(raw[f])

        df = pd.DataFrame([[X[f] for f in CORRECT_ORDER]], columns=CORRECT_ORDER)
        pred = model.predict(df)[0]
        prob = round(model.predict_proba(df)[0][1], 3)

        result = "❌ Customer WILL Churn" if pred == 1 else "✔ Customer will NOT Churn"

        return render_template("result.html", result=result, prob=prob)

    except Exception as e:
        return render_template("result.html", result=f"Error: {e}", prob=None)

if __name__ == "__main__":
    print("🚀 Server Started...")
    app.run(host="0.0.0.0", port=8000)


##  Cell: Home Page UI for Customer Churn Prediction (home.html)

This cell creates the **landing page (home screen)** for your **Telecom Customer Churn Prediction Web App**.  
It provides:

-  A clean project title
-  A short project description
-  A **Call-To-Action button** to start prediction
-  A modern and minimal user interface entry point

---

## ✅Purpose of `home.html`

This page acts as:

- 🏠 The **first page users see**
- 🚀 The **starting point for prediction**
- 🎯 A **professional project showcase interface**

Users click the **“Start Prediction 🚀”** button to move to the input form.

---

##  HTML Structure Explanation

###  Page Wrapper


In [ ]:
%%writefile templates/home.html
<body class="home-body">

<div class="home-box">
    <h1 class="home-title">📊 Telecom Churn Predictor</h1>
    <p class="home-subtitle">A clean and modern UI to predict churn with accuracy.</p>

    <a href="/predict" class="home-btn">Start Prediction 🚀</a>
</div>

</body>


##  Cell: Customer Input Form UI (input.html)

This cell creates the **main data entry form** for your **Telecom Customer Churn Prediction Web App**.  
It allows users to enter **all customer details required by the ML model** in a clean, structured interface.

---

##  Purpose of `input.html`

This page is responsible for:

-  Collecting **customer service & usage details**
-  Displaying **dropdowns for categorical features**
-  Validating **required inputs**
-  Sending data to the Flask `/predict` route for ML inference
-  Displaying validation errors if fields are missing

---

##  Technologies Used

- **HTML5** → Page structure  
  🔗 https://developer.mozilla.org/en-US/docs/Web/HTML  
- **CSS3** → Styling  
  🔗 https://developer.mozilla.org/en-US/docs/Web/CSS  
- **Flask Jinja Templates** → Dynamic values from backend  
  🔗 https://flask.palletsprojects.com/en/latest/templating/

---

In [ ]:
%%writefile templates/input.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>🧾 Customer Details Input</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
  <div class="container-wide">
    <div class="glass-card">
      <h2>Enter Customer Details</h2>

      {% if error %}
      <div class="error">{{ error }}</div>
      {% endif %}

      <form method="POST" action="/predict">

        <div class="two-col">
          <div>
            <label>State *</label>
            <select name="State" required>
              {% for s in states %}
              <option value="{{ s }}">{{ s }}</option>
              {% endfor %}
            </select>

            <label>Account Length *</label>
            <input type="number" step="any" name="Account length" required>

            <label>Area Code *</label>
            <select name="Area code" required>
              {% for a in area_codes %}
              <option value="{{ a }}">{{ a }}</option>
              {% endfor %}
            </select>

            <label>International Plan *</label>
            <select name="International plan" required>
              {% for y in yes_no %}
              <option value="{{ y }}">{{ y }}</option>
              {% endfor %}
            </select>

            <label>Voice Mail Plan *</label>
            <select name="Voice mail plan" required>
              {% for y in yes_no %}
              <option value="{{ y }}">{{ y }}</option>
              {% endfor %}
            </select>
          </div>

          <div>
            <label>Number Vmail Messages *</label>
            <input type="number" step="any" name="Number vmail messages" required>

            <label>Total Day Minutes *</label>
            <input type="number" step="any" name="Total day minutes" required>

            <label>Total Day Calls *</label>
            <input type="number" step="any" name="Total day calls" required>

            <label>Total Day Charge *</label>
            <input type="number" step="any" name="Total day charge" required>

            <label>Total Eve Minutes *</label>
            <input type="number" step="any" name="Total eve minutes" required>

            <label>Total Eve Calls *</label>
            <input type="number" step="any" name="Total eve calls" required>

            <label>Total Eve Charge *</label>
            <input type="number" step="any" name="Total eve charge" required>

            <label>Total Night Minutes *</label>
            <input type="number" step="any" name="Total night minutes" required>

            <label>Total Night Calls *</label>
            <input type="number" step="any" name="Total night calls" required>

            <label>Total Night Charge *</label>
            <input type="number" step="any" name="Total night charge" required>

            <label>Total Intl Minutes *</label>
            <input type="number" step="any" name="Total intl minutes" required>

            <label>Total Intl Calls *</label>
            <input type="number" step="any" name="Total intl calls" required>

            <label>Total Intl Charge *</label>
            <input type="number" step="any" name="Total intl charge" required>

            <label>Customer Service Calls *</label>
            <input type="number" step="any" name="Customer service calls" required>
          </div>
        </div>

        <button class="btn" type="submit">🔍 Predict</button>
        <a class="btn-alt btn" href="/">Home</a>
      </form>

    </div>
  </div>
</body>
</html>


## 🟢 Cell: Prediction Result UI (result.html)

This cell creates the **final output page** of your **Telecom Customer Churn Prediction Web App**.  
It displays:

- ✅ The **churn prediction result**
- ✅ The **probability/confidence score**
- ✅ Navigation options to:
  - Enter another customer
  - Return to the home page

---

##  Purpose of `result.html`

This page is responsible for:

-  Showing the **final ML prediction**
-  Displaying the **model’s confidence score**
-  Providing **user-friendly navigation**
-  Acting as the **final stage of the prediction pipeline**

---

## ✅ Technologies Used

- **HTML5** → Page structure  
  🔗 https://developer.mozilla.org/en-US/docs/Web/HTML  
- **CSS3** → Styling  
  🔗 https://developer.mozilla.org/en-US/docs/Web/CSS  
- **Flask Jinja Templates** → Dynamic prediction values  
  🔗 https://flask.palletsprojects.com/en/latest/templating/

---

In [ ]:
%%writefile templates/result.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>📊 Prediction Result</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
  <div class="container-wide">
    <div class="glass-card">
      <h2>Prediction Outcome</h2>

      <p class="result-text">{{ result }}</p>

      {% if prob is not none %}
      <p class="prob">Probability: <strong>{{ prob }}</strong></p>
      {% endif %}

      <a class="btn" href="/input">Enter Another</a>
      <a class="btn-alt btn" href="/">Home</a>
    </div>
  </div>
</body>
</html>


##  Cell: Global Styling & UI Design (static/style.css)

This cell defines the **complete visual design system** for your  
**Telecom Customer Churn Prediction Web App**.

It controls:

-  Page layout & alignment  
-  Glassmorphism card design  
-  Form input styling  
-  Buttons & hover effects  
-  Result & error message styling  
-  Home page center alignment  
-  Responsive layout  
-  Smooth animations  

---

##  Purpose of `style.css`

This file ensures that your web application:

✔ Looks **modern and professional**  
✔ Has a **clean medical-grade UI feel**  
✔ Is **fully responsive**  
✔ Provides **excellent user experience (UX)**  
✔ Feels like a **real production AI application**

---

##  Core Technologies Used

- **CSS3 (Cascading Style Sheets)**  
  🔗 https://developer.mozilla.org/en-US/docs/Web/CSS  

- **Glassmorphism UI Design Concept**  
  🔗 https://uxdesign.cc/glassmorphism-in-user-interfaces-1f39bb1308c9  

- **Flexbox & Grid Layouts**  
  🔗 https://developer.mozilla.org/en-US/docs/Learn/CSS/CSS_layout  

---

In [ ]:
%%writefile static/style.css
/* ------------------------------------------
   GLOBAL STYLING — LIGHT, MODERN, MINIMAL
-------------------------------------------*/
body {
  margin: 0;
  padding: 0;
  font-family: 'Inter', sans-serif;
  background: linear-gradient(135deg, #e9f2f8, #f6fbff);
  color: #333;
  display: flex;
  justify-content: center;
  align-items: flex-start;
  padding-top: 50px;
  min-height: 100vh;
}

/* Wrapper */
.container-wide {
  width: 100%;
  max-width: 900px;
  padding: 0 20px;
}

/* ------------------------------------------
   GLASS CARD (Main container)
-------------------------------------------*/
.glass-card {
  background: rgba(255, 255, 255, 0.65);
  border-radius: 20px;
  padding: 40px 50px;
  backdrop-filter: blur(18px);
  -webkit-backdrop-filter: blur(18px);
  box-shadow: 0 18px 50px rgba(0, 0, 0, 0.1);
  border: 1px solid rgba(255, 255, 255, 0.6);
  animation: fadeIn 0.7s ease-out;
}

/* HEADINGS */
h1 {
  font-size: 28px;
  font-weight: 700;
  color: #1e72c5;
  margin-bottom: 10px;
  text-align: center;
}

h2 {
  font-size: 20px;
  font-weight: 600;
  color: #4a4a4a;
  margin-bottom: 25px;
  text-align: center;
}

/* ------------------------------------------
   FORM INPUTS
-------------------------------------------*/
label {
  display: block;
  margin-top: 18px;
  font-size: 15px;
  font-weight: 500;
  color: #4c4c4c;
}

input, select {
  width: 100%;
  padding: 14px;
  margin-top: 6px;
  border-radius: 12px;
  border: 1px solid #d5dce3;
  background: #fdfdfd;
  font-size: 15px;
  transition: 0.25s ease;
}

input:focus,
select:focus {
  border-color: #6bb8ff;
  box-shadow: 0 0 8px rgba(107, 184, 255, 0.4);
  outline: none;
}

/* ------------------------------------------
   BUTTONS
-------------------------------------------*/
.btn {
  display: inline-block;
  width: 100%;
  padding: 15px;
  margin-top: 25px;
  font-size: 17px;
  font-weight: 600;
  border-radius: 12px;
  border: none;
  color: white;
  cursor: pointer;
  background: linear-gradient(135deg, #4facfe, #00f2fe);
  transition: 0.3s ease;
}

.btn:hover {
  transform: translateY(-3px);
  box-shadow: 0 8px 20px rgba(79, 172, 254, 0.3);
}

/* Secondary button */
.btn-alt {
  background: #ffffff;
  color: #333;
  border: 1px solid #d7d7d7;
}

.btn-alt:hover {
  background: #f5f5f5;
}

/* ------------------------------------------
   RESULT BOX
-------------------------------------------*/
.result-box {
  text-align: center;
  margin-top: 25px;
  padding: 20px;
  background: rgba(240, 248, 255, 0.8);
  border-radius: 12px;
  border-left: 5px solid #4facfe;
  animation: fadeIn 0.8s ease-out;
}

.result-text {
  font-size: 24px;
  font-weight: 700;
  color: #1e72c5;
}

.prob {
  font-size: 18px;
  color: #555;
  margin-top: 6px;
}

/* ------------------------------------------
   ERROR BOX
-------------------------------------------*/
.error {
  background: #ffe9e9;
  border-left: 4px solid #e05252;
  color: #a03333;
  padding: 12px;
  border-radius: 8px;
  margin-bottom: 20px;
}

/* ------------------------------------------
   HOME PAGE — PERFECT CENTER ALIGNMENT
-------------------------------------------*/
.home-body {
  margin: 0;
  padding: 0;
  height: 100vh;
  width: 100vw;

  display: flex;
  justify-content: center;
  align-items: center;

  background: linear-gradient(135deg, #e7f1fa, #f7fbff);
  font-family: 'Inter', sans-serif;
}

/* Home Card */
.home-box {
  width: 450px;
  padding: 40px;
  border-radius: 20px;
  background: rgba(255, 255, 255, 0.65);
  backdrop-filter: blur(15px);
  -webkit-backdrop-filter: blur(15px);
  border: 1px solid rgba(255, 255, 255, 0.7);

  text-align: center;
  box-shadow: 0 15px 45px rgba(0,0,0,0.08);
  animation: fadeIn 0.8s ease-out;
}

/* Header + Subheading */
.home-title {
  font-size: 32px;
  font-weight: 700;
  color: #146cc3;
  margin-bottom: 10px;
}

.home-subtitle {
  font-size: 17px;
  margin-bottom: 25px;
  color: #5a5a5a;
}

/* Button */
.home-btn {
  margin-top: 20px;
  padding: 14px 22px;
  display: inline-block;
  font-size: 17px;
  font-weight: 600;
  text-decoration: none;
  border-radius: 10px;
  background: linear-gradient(135deg, #4facfe, #00f2fe);
  color: white;
  transition: 0.3s ease;
}

.home-btn:hover {
  transform: scale(1.05);
  box-shadow: 0 10px 25px rgba(79, 172, 254, 0.35);
}

/* ------------------------------------------
   RESPONSIVE
-------------------------------------------*/
@media (min-width: 768px) {
  form .two-col {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 20px;
  }
}

/* Fade animation */
@keyframes fadeIn {
  from { opacity: 0; transform: translateY(12px); }
  to { opacity: 1; transform: translateY(0); }
}


###  Cell : Stop Any Previously Running Flask & ngrok Processes

This cell safely **terminates any existing Flask servers and ngrok tunnels** before starting a new deployment instance.

---

####  Why This Step Is Important

If old processes are still running:
-  Flask may fail to start due to **port already in use**
-  ngrok may create **multiple conflicting tunnels**
-  You may see incorrect or cached outputs

Stopping them ensures:
 Clean server restart  
 Reliable ngrok tunnel creation  
 No port conflicts  



In [ ]:
# ===============================
# 6️⃣ Kill any previous processes
# ===============================
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"




### 🟢 Cell : Check If Port 5000 Is Already in Use

This cell checks whether **any process is currently using port 8000**, which is the default port for your Flask web application.

---

#### ✅ Why This Step Is Important

If port `8000` is already in use:
-  Flask will fail to start  
-  You may see an error like: *“Address already in use”*  
-  Your app will not be accessible  

This command helps you **identify which process is blocking the port** before launching Flask.



In [ ]:
!lsof -i :8000

### 🟢 Cell : Forcefully Terminate a Specific Process by PID

This cell forcefully **kills a specific running process** using its **Process ID (PID)**.  
It is typically used when a process is **locking the Flask port (8000)** and cannot be stopped normally.

---

#### ✅ Why This Step Is Important

If a Flask or background process:
-  Does not stop using `pkill`
-  Continues to occupy port `8000`
-  Prevents your web app from launching  

Then force-killing the process by PID ensures:
 Immediate termination  
 Port is released instantly  
 No further port conflicts  

In [ ]:


!kill -9 4558

##### Running Flask App

In [ ]:
# ===============================
# 7️⃣ Run Flask in the background
# ===============================
!nohup python app.py > flask.log 2>&1 &

### 🔑 How to Create & Use an ngrok Authentication Token (One-Time Setup)

This step is **mandatory** to generate a public URL for your Flask app running in Google Colab.

---

####  Step 1: Create a Free ngrok Account

1. Open the official ngrok website:  
   https://ngrok.com  
2. Click **Sign Up**  
3. Sign up using:
   - Google account OR
   - Email + password  
4. Log in to your ngrok dashboard after signup.

---

####  Step 2: Get Your ngrok Auth Token

1. After logging in, go to:  
   **Dashboard → Your Authtoken**
2. You will see a command like this:

In [ ]:
# ===============================
# 8️⃣ Start ngrok tunnel
# ===============================
from pyngrok import ngrok, conf
conf.get_default().auth_token = "replace with your token"  # 🔑 replace with your token

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# ===============================
# 9️⃣ Check logs (optional)
# ===============================
!sleep 3 && tail -n 20 flask.log

In [ ]:
!tail -n 50 flask.log